In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn import metrics

import torch # Libreria
from torch.utils.data import Dataset # Clase u objeto que va a contener la informacion que vamos a utilizar para entrenar y evaluar nuestro algoritmo
from torch.utils.data import DataLoader

## Uso de GPU

In [ ]:
device = ""
if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"

print(torch.cuda.is_available())

In [ ]:
if torch.cuda.is_available():
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)

In [ ]:
print("Device: ",device)

## CustomDataset y DataLoader

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = x
        self.y = y

    def __len__(self):
        return self.x.shape[0]
  
    def __getitem__(self, idx):
        return self.x[idx,:], self.y[idx]

In [ ]:
training_set = CustomDataset(x_train, l_train)
print(training_set.__len__())
print(training_set.__getitem__(0))

In [ ]:
test_set = CustomDataset(x_test, l_test)
print(test_set.__len__())
print(test_set.__getitem__(0))

El **DataLoader** simpre espera el set de datos, el batch_size que preferentemente deberia ser potencia de 2 para optimizar los calculos, y opcional el shuffel que mezcla los datos cada vez que comienza una epoch

In [ ]:
training_dataloader = DataLoader(training_set, batch_size = 512, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size = 256, shuffle=True)

In [ ]:
len(training_dataloader) # Muestra el tamaño de cada batch

## Regresion Logistica Bivariada

In [ ]:
# Definimos el modelo
class NNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_1 = torch.nn.Linear(in_features = 2, out_features = 10, bias = True)
        self.sigmoid_1 = torch.nn.ReLU()
        self.linear_2 = torch.nn.Linear(in_features = 10, out_features = 20, bias = True)
        self.sigmoid_2 = torch.nn.ReLU()
        self.linear_3 = torch.nn.Linear(in_features = 20, out_features = 1, bias = True)
        self.sigmoid_3 = torch.nn.ReLU()

    def forward(self, x):
                                    # x.shape = 512 x 2
        z1 = self.linear_1(x)       # z1.shape = 512 x 10
        a1 = self.relu_1(z1)        # a1.shape = 512 x 10
        z2 = self.linear_2(a1)      # z2.shape = 512 x 20 
        a2 = self.relu_2(z2)        # a2.shape = 512 x 20
        z3 = self.linear_3(a2)      # z3.shape = 512 x 1 -> [-inf, +inf]
        y = self.relu_3(z3)         # y.shape = 512 x 1 -> [0, 1]
        return y

In [ ]:
nnet = NNet()
print(nnet)

In [ ]:
criterion = torch.nn.BCELoss(reduction='sum') # criterion, is my lost fuction
optimizer = torch.optim.SGD(nnet.parameters(), lr=0.005) # is my optimizer

In [ ]:
nnet.to(device)

In [ ]:
for epoch in range(100):
    running_loss = 0
    for i, data in enumerate(training_dataloader):
        # data
        x, y = data
        x = x.to(device).float()
        y = y.to(device).float().reshape(-1,1)

        # set gradient to zero
        optimizer.zero_grad()  #pytorch requiere que inicializemos en cada corrida los gradientes

        #forward
        y_hat = nnet(x)

        #loss
        loss = criterion(y_hat, y)

        #backward
        loss.backward()

        #update of parameters
        optimizer.step()

        #compute metrics and statistics
        running_loss += loss.item()
  
    print(f"Epoch = {epoch} - loss = {running_loss / len(training_set)}")

In [ ]:
for i, data in enumerate(test_dataloader):
    # compute metrics
    # precision, recall,  acc, f1
    # use scikit learn
  
    # data
    x_tst, y_tst = data
    x_tst = x_tst.to(device).float()
    #y_tst = y_tst.to(device).float().reshape(-1,1)

    y_hat_test = (nnet(x_tst).detach().numpy() >= 0.5)

    print("Accuracy:",metrics.accuracy_score(y_tst, y_hat_test))
    print("Precision:",metrics.precision_score(y_tst, y_hat_test))
    print("Recall:",metrics.recall_score(y_tst, y_hat_test))
    print("F1:",metrics.f1_score(y_tst, y_hat_test))
    print()

## Regresion Lineal